In [ ]:
#!pip install azureml-opendatasets
#!pip install gremlinpython

In [ ]:
# This is a package in preview.
from azureml.opendatasets import PublicHolidays

from datetime import datetime

end_date = datetime(2021,12,31)
start_date = datetime(2021,12,1)
hol = PublicHolidays(start_date=start_date, end_date=end_date)
hol_df = hol.to_pandas_dataframe()

In [ ]:
from gremlin_python.driver import client
# Make sure to create your Cosmos DB Gremlin API endpoint at https://portal.azure.com
# Create the Database and Collection in the portal
# This script will populate the data that we use in our demo
ENDPOINT = 'YOUR-GREMLIN-API-HOST.gremlin.cosmosdb.azure.com'
PRIMARY_KEY = 'YOUR-GREMLIN-API-READ-WRITE-KEY'
DATABASE = 'graphmas'
COLLECTION = 'holidays'

In [ ]:
# Build the vertices - countries, holidays
VERTICES = []
EDGES = []
countries = hol_df.countryOrRegion.unique()
for country in countries:
    VERTICES.append(f"g.addV('Country').property('name','{country}')")

In [ ]:
holidays = hol_df[['normalizeHolidayName','date']].drop_duplicates()
for index,holiday in holidays.iterrows():
    VERTICES.append(f"g.addV('Holiday').property('name',\"{holiday['normalizeHolidayName']}\").property('holidayDate',\"{holiday['date']}\")")

In [ ]:
# Build the edges
for index, holiday in hol_df.iterrows():
    EDGES.append(f"g.V().has('name',\"{holiday['normalizeHolidayName']}\").has('holidayDate','{holiday['date']}').addE('celebrates').from(g.V().hasLabel('Country').has('name','{holiday['countryOrRegion']}'))")    

In [ ]:
# Need the nest_asyncio to help with Jupyter Notebook handling this
import nest_asyncio
nest_asyncio.apply()

# Now let's do this
# This code is based off of this blog post: https://www.taygan.co/blog/2018/01/23/azure-cosmos-db-graph-api-with-python
from gremlin_python.driver import client, serializer

def cleanup_graph(gremlin_client):    
    callback = gremlin_client.submitAsync("g.V().drop()")
    if callback.result() is not None:
        print("Cleaned up the graph!")

def insert_vertices(gremlin_client):
    for vertex in VERTICES:
        callback = gremlin_client.submitAsync(vertex)
        if callback.result() is None:            
            print("Something went wrong with this query: {0}".format(vertex))

def insert_edges(gremlin_client):
    for edge in EDGES:
        callback = gremlin_client.submitAsync(edge)
        if callback.result() is None:            
            print("Something went wrong with this query:\n{0}".format(edge))

def handler():
    # Initialise client
    print('Initializing client...')
    # GraphSON V2 is called out here, as V3 is not supported yet
    gremlin_client = client.Client(
        'wss://' + ENDPOINT + ':443/', 'g',
        username="/dbs/" + DATABASE + "/colls/" + COLLECTION,
        password=PRIMARY_KEY,
        message_serializer=serializer.GraphSONSerializersV2d0()
    )
    print('Client initialised!')

    # Purge graph
    cleanup_graph(gremlin_client)

    # Insert vertices (i.e. nodes)
    insert_vertices(gremlin_client)

    # Insert edges (i.e. nodes)
    insert_edges(gremlin_client)

    print('Finished!')

if __name__ == '__main__':
    handler()
